In [0]:
DECLARE OR REPLACE VARIABLE var_recent_ts TIMESTAMP;
SET VAR var_recent_ts =
(SELECT MAX(load_timestamp) recent_ts
FROM default.sales_brnz);

In [0]:
CREATE OR REPLACE TEMP VIEW sales_country_temp_v AS
SELECT DISTINCT country, region, var_recent_ts as recent_timestamp
FROM default.sales_brnz
WHERE load_timestamp = var_recent_ts

In [0]:
MERGE INTO country_dim_slvr t
USING sales_country_temp_v s
ON t.country = s.country
WHEN MATCHED THEN UPDATE SET t.region = s.region, t.recent_load_ts = s.recent_timestamp
WHEN NOT MATCHED THEN INSERT (country, region, recent_load_ts) VALUES (s.country, s.region, s.recent_timestamp);

In [0]:
CREATE OR REPLACE TEMP VIEW sales_item_type_temp_v AS
SELECT DISTINCT item_type, var_recent_ts as recent_timestamp
FROM default.sales_brnz
WHERE load_timestamp = var_recent_ts

In [0]:
MERGE INTO item_type_dim_slvr t
USING sales_item_type_temp_v s
ON t.item_type = s.item_type
WHEN NOT MATCHED THEN INSERT (item_type, recent_load_ts) VALUES (s.item_type, s.recent_timestamp)

In [0]:
CREATE OR REPLACE TEMP VIEW sales_order_type_temp_v AS
SELECT DISTINCT sales_channel, order_priority, var_recent_ts as recent_timestamp
FROM default.sales_brnz
WHERE load_timestamp = var_recent_ts

In [0]:
MERGE INTO order_type_dim_slvr t
USING sales_order_type_temp_v s
ON t.sales_channel = s.sales_channel AND t.order_priority = s.order_priority
WHEN NOT MATCHED THEN INSERT (sales_channel, order_priority, recent_load_ts) VALUES (s.sales_channel, s.order_priority, s.recent_timestamp)